In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sb

from functools import reduce
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
import statsmodels.formula.api as sm
from sklearn.metrics import confusion_matrix

In [ ]:
# Read Data

acquisition = ['LoanID','Channel','SellerName','OrigInterestRate','OrigUnpPrinc','OrigLoanTerm',
               'OrigDate','FirstPayment','OrigLTV','OrigCLTV','NumBorrow','DTIRat','CreditScore',
               'FTHomeBuyer','LoanPurpose','PropertyType','NumUnits','OccStatus','PropertyState',
               'Zip','MortInsPerc','ProductType','CoCreditScore','MortInsType','RelMortInd']

performance = ['LoanID','MonthRep','Servicer','LAST_RT','LAST_UPB',
               'LoanAge','MonthsToMaturity','AdMonthsToMaturity','MaturityDate','MSA',
               'CLDS','ModFlag','ZB_Code','ZB_Date','LPI_Date',
               'FCC_Date','DISP_Date','FCC_COST','PP_COST','AR_COST',
               'IE_COST', 'TAX_COST','NS_PROCS','CE_PROCS','RMW_PROCS','O_PROCS','NON_INT_UPB',
              'PRIN_FORG_UPB_FHFA','REPCH_FLAG','PRIN_FORG_UPB_OTH','TRANSFER_FLAG']

a1 = pd.read_csv('C:/Class/cecl/Acquisition_2006Q1.txt', sep='|', names=acquisition, index_col=False)
p1 = pd.read_csv('C:/Class/cecl/Performance_2006Q1.txt', sep='|', names=performance, index_col=False)


In [3]:
keep_var_acq = ['LoanID','OrigInterestRate','OrigUnpPrinc','OrigLTV','CreditScore']
keep_var_per = ['LoanID','MonthRep','LAST_RT','LAST_UPB',
               'MonthsToMaturity','CLDS',
               'ZB_Code','ZB_Date','LPI_Date','FCC_Date','DISP_Date',
               'FCC_COST','PP_COST','AR_COST','IE_COST','TAX_COST','NS_PROCS',
               'O_PROCS','NON_INT_UPB']

a1_sub = a1[keep_var_acq].reset_index(drop=True)
p1_sub = p1[keep_var_per].reset_index(drop=True)

a1 = a1_sub.dropna(subset = ['CreditScore'])


In [4]:
def_info = p1_sub.loc[p1_sub['ZB_Code'].isin([2.0,3.0,6.0,9.0,15,16])]

In [5]:
prepay_info = p1_sub.loc[p1_sub['ZB_Code'].isin([1.0])]

In [6]:
sample_id = a1[~a1['LoanID'].isin(def_info['LoanID'])].sample(n=def_info.shape[0],random_state=1)

In [7]:
sample_a1 = a1.loc[a1['LoanID'].isin(sample_id['LoanID']) | a1['LoanID'].isin(def_info['LoanID'])]

In [8]:
sample_p1 = p1_sub.loc[p1['LoanID'].isin(sample_a1['LoanID'])]

In [9]:
sample_p1 = sample_p1.assign(**{'prepay_flag':np.nan})

In [10]:
sample_p1.loc[sample_p1['LoanID'].isin(prepay_info['LoanID']),'prepay_flag']=1

In [11]:
def f_delq(row):
    if row['prepay_flag'] == 1:
        if row['CLDS'] == '0':
            return 0
        elif row['ZB_Code'] == 1.0:
            return -1
    else:
        if row['CLDS'] == '0':
            return 0
        elif row['CLDS'] == '1':
            return 1
        elif row['CLDS'] == '2':
            return 2
        elif row['CLDS'] == '3':
            return 3
        elif row['ZB_Code'] in [2.0,3.0,6.0,9.0,15,16]:
            return 4

In [12]:
sample_p1 = sample_p1.assign(DELQ_STAT= sample_p1.apply(f_delq, axis=1))

In [13]:
# Define Transitions

In [14]:
sample_p1 = sample_p1.dropna(subset=["DELQ_STAT"])

In [15]:
first_row_index = sample_p1.groupby(['LoanID'], as_index=False).apply(lambda g: g.index[0])

In [16]:
sample_p1.loc[first_row_index,'firstrow'] = 1

In [17]:
sample_p1['delq_shift'] = sample_p1.DELQ_STAT.shift()

In [18]:
def f_tran(row): # transition
    if row['firstrow'] != 1:
        if row['DELQ_STAT'] == 0 and row['delq_shift'] == 0:
            return '00'
        elif row['DELQ_STAT'] == 1 and row['delq_shift'] == 0:
            return '10'
     #   elif row['DELQ_STAT'] == 1 and row['delq_shift'] == 1:
      #      return '11'
        elif row['DELQ_STAT'] == 0 and row['delq_shift'] == 1:
            return '01'
        elif row['DELQ_STAT'] == 2 and row['delq_shift'] == 1:
            return '12'
        elif row['DELQ_STAT'] == 1 and row['delq_shift'] == 2:
            return '21'
    #    elif row['DELQ_STAT'] == 2 and row['delq_shift'] == 2:
     #       return '22'
        elif row['DELQ_STAT'] == 3 and row['delq_shift'] == 2:
            return '23'
        elif row['DELQ_STAT'] == 2 and row['delq_shift'] == 3:
            return '32'
    #    elif row['DELQ_STAT'] == 3 and row['delq_shift'] == 3:
     #       return '33'
        elif row['DELQ_STAT'] == 4 and row['delq_shift'] == 3:
            return '34'
        elif row['DELQ_STAT'] == -1 and row['delq_shift'] == 0:
            return '0-1'

In [19]:
sample_p1 = sample_p1.assign(tran= sample_p1.apply(f_tran, axis=1))

In [20]:
# Read Historical Macro Economic Variables (MEV)
unemployment = pd.read_csv('/Users/xy4/Documents/CECL/UNRATE.csv', sep=',', names=['DATE','UNEMPLOY'],index_col=False)
unemployment['DATE'] = pd.to_datetime(unemployment.DATE)
unemployment['DATE'] = unemployment['DATE'].dt.strftime('%m/%d/%Y')
hpi = pd.read_csv('/Users/xy4/Documents/CECL/HPI.csv', sep=',', names=['DATE','HPI'], index_col=False)
hpi['DATE'] = pd.to_datetime(hpi.DATE)
hpi['DATE'] = hpi['DATE'].dt.strftime('%m/%d/%Y')
cpi = pd.read_csv('/Users/xy4/Documents/CECL/CPALTT01USM661S.csv', sep=',', names=['DATE','CPALTT01USM661S'], index_col=False)
cpi['DATE'] = pd.to_datetime(cpi.DATE)
cpi['DATE'] = cpi['DATE'].dt.strftime('%m/%d/%Y')
gs3m = pd.read_csv('/Users/xy4/Documents/CECL/GS3M.csv', sep=',', names=['DATE','GS3M'], index_col=False) # 3 month treasure rate
gs3m['DATE'] = pd.to_datetime(gs3m.DATE)
gs3m['DATE'] = gs3m['DATE'].dt.strftime('%m/%d/%Y')
gs5 = pd.read_csv('/Users/xy4/Documents/CECL/GS5.csv', sep=',', names=['DATE','GS5'], index_col=False) # 3 month treasure rate
gs5['DATE'] = pd.to_datetime(gs5.DATE)
gs5['DATE'] = gs5['DATE'].dt.strftime('%m/%d/%Y')
gs10 = pd.read_csv('/Users/xy4/Documents/CECL/GS5.csv', sep=',', names=['DATE','GS10'], index_col=False) # 3 month treasure rate
gs10['DATE'] = pd.to_datetime(gs10.DATE)
gs10['DATE'] = gs10['DATE'].dt.strftime('%m/%d/%Y')
# Make one table
temp = [unemployment,hpi,cpi,gs3m,gs5,gs10]
MEV = reduce(lambda left,right:pd.merge(left,right,on='DATE'),temp)

In [21]:
MEV.set_index('DATE',inplace=True)

In [22]:
MEV = MEV.pct_change()

In [23]:
mQ1 = pd.merge(sample_a1,sample_p1,how="outer", on=['LoanID'])

In [24]:
mQ1 = pd.merge(mQ1, MEV, how = 'left', left_on = "MonthRep", right_on="DATE")

In [25]:
mQ1 = mQ1.dropna(subset=['tran'])

In [26]:
mQ1_00 = mQ1.loc[mQ1['tran'] == '00']

In [27]:
mQ1_000 = mQ1.loc[mQ1['tran'] != '00']

In [28]:
mQ1_00_sample = mQ1_00.sample(frac=0.01,random_state=1)

In [29]:
mQ1 = pd.concat([mQ1_000, mQ1_00_sample], join="inner")

In [30]:
MEVs = ['UNEMPLOY','HPI','GS3M','CreditScore']


In [31]:
# Regression for state 0
mQ1_0 = mQ1.loc[mQ1['tran'].isin(['00','01','0-1'])]
X = pd.DataFrame(mQ1_0,columns=MEVs).values
Y = mQ1_0['tran'].values

In [32]:
X_train0, X_test0, Y_train0, Y_test0 = train_test_split(X,Y, test_size = 0.3, random_state = 1)
LogReg0 = LogisticRegression()
state_0 = LogReg0.fit(X_train0,Y_train0)
Y_pred0 = LogReg0.predict(X_test0)
print(classification_report(Y_test0,Y_pred0))

             precision    recall  f1-score   support

        0-1       0.46      0.58      0.51      5711
         00       0.38      0.07      0.12      5882
         01       0.54      0.78      0.64      7487

avg / total       0.47      0.50      0.44     19080



In [33]:
# Regression for state 1
mQ1_1 = mQ1.loc[mQ1['tran'].isin(['10','12'])]
X = pd.DataFrame(mQ1_1,columns=MEVs).values
Y = mQ1_1['tran'].values

In [34]:
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X,Y, test_size = 0.3, random_state = 1)
LogReg1 = LogisticRegression()
state_1 = LogReg1.fit(X_train1,Y_train1)
Y_pred1 = LogReg1.predict(X_test1)
print(classification_report(Y_test1,Y_pred1))

             precision    recall  f1-score   support

         10       0.59      1.00      0.74     16455
         12       0.00      0.00      0.00     11268

avg / total       0.35      0.59      0.44     27723



C:\Users\xy4\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:
mQ1_2 = mQ1.loc[mQ1['tran'].isin(['21','23'])]
X = pd.DataFrame(mQ1_2,columns=MEVs).values
Y = mQ1_2['tran'].values

In [36]:
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X,Y, test_size = 0.3, random_state = 1)
LogReg2 = LogisticRegression()
state_2 = LogReg2.fit(X_train2,Y_train2)
Y_pred2 = LogReg2.predict(X_test2)
print(classification_report(Y_test2,Y_pred2))

             precision    recall  f1-score   support

         21       0.00      0.00      0.00      1579
         23       0.85      1.00      0.92      9133

avg / total       0.73      0.85      0.78     10712



C:\Users\xy4\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [37]:
mQ1_3 = mQ1.loc[mQ1['tran'].isin(['32','34'])]
X = pd.DataFrame(mQ1_3,columns=MEVs).values
Y = mQ1_3['tran'].values

In [38]:
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X,Y, test_size = 0.3, random_state = 1)
LogReg3 = LogisticRegression()
state_3 = LogReg3.fit(X_train3,Y_train3)
Y_pred3 = LogReg3.predict(X_test3)
print(classification_report(Y_test3,Y_pred3))

             precision    recall  f1-score   support

         32       0.00      0.00      0.00       636
         34       0.91      1.00      0.95      6079

avg / total       0.82      0.91      0.86      6715



C:\Users\xy4\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
# LGD

In [40]:
lgd_data = pd.merge(a1.loc[a1['LoanID'].isin(def_info['LoanID'])],
                    p1.loc[p1['ZB_Code'].isin([2.0,3.0,6.0,9.0,15,16])],
                           how="outer", on=['LoanID'])

In [41]:
lgd_data = lgd_data.dropna(subset=['CreditScore'])

In [42]:
lgd_data = pd.merge(lgd_data,MEV,how = 'left', left_on = "MonthRep", right_on="DATE")

In [43]:
lgd_data['Expenses'] =  lgd_data[['FCC_COST','PP_COST','AR_COST',
               'IE_COST']].sum(axis=1)

In [44]:
lgd_data['Proceeds'] = lgd_data[['NS_PROCS','CE_PROCS','RMW_PROCS','O_PROCS']].sum(axis=1)

In [45]:
lgd_data['Loss'] = lgd_data['LAST_UPB'] + lgd_data['Expenses']-lgd_data['Proceeds']

In [46]:
lgd_data.loc[lgd_data['Loss']<0]=0

In [47]:
lgd_data['Y'] = lgd_data['Loss']/lgd_data['LAST_UPB']

In [208]:
lgd_result = sm.ols(formula="Y ~ CreditScore + HPI + UNEMPLOY", data=lgd_data).fit()

In [50]:
last_row_indicator = p1.groupby(['LoanID'], as_index=False).apply(lambda g: g.index[-1])

In [131]:
p1.loc[last_row_indicator,'lastrow'] = 1

In [132]:
pd_data = p1.loc[p1['lastrow']==1]

In [133]:
pd_data = pd_data[~pd_data['ZB_Code'].isin([1.0,2.0,3.0,6.0,9.0,15,16])]

In [134]:
pd_data = pd_data.dropna(axis='columns')

In [135]:
pd_data = pd_data.drop(columns = ['lastrow','TRANSFER_FLAG','ModFlag','MSA'])

In [163]:
pd_mev = pd.merge(pd_data, MEV, how = 'left', left_on = "MonthRep", right_on="DATE")

In [144]:
cresco = a1[['LoanID','CreditScore']]

In [164]:
pd_mev = pd.merge(pd_mev, cresco, how = 'left', left_on = "LoanID", right_on="LoanID")

In [165]:
pd_mev = pd_mev.dropna(subset=['CreditScore'])

In [166]:
pd_mev = pd_mev.reset_index(drop=True)

In [170]:
pred0 = state_0.predict_proba(pd_mev[MEVs].values)
pred1 = state_1.predict_proba(pd_mev[MEVs].values)
pred2 = state_2.predict_proba(pd_mev[MEVs].values)
pred3 = state_3.predict_proba(pd_mev[MEVs].values)
pd_mev['DefaultF'] = 0
pd_mev['DefaultP'] = 0
pd_mev['DefaultS'] = 0

In [172]:
for i, row in pd_mev.iterrows():
    a = pred0[i]
    b = pred1[i]
    c = pred2[i]
    d = pred3[i]
    TPM = np.array([[1,0,0,0,0,0],
                  [a[0],a[1],a[2],0,0,0],
                  [0,b[0],0,b[1],0,0,],
                  [0,0,c[0],0,c[1],0],
                  [0,0,0,d[0],0,d[1]],
                  [0,0,0,0,0,1]])
    j = 1
    P = TPM
    if pd_mev.iloc[i,7] == '0':
        k = 1
    elif pd_mev.iloc[i,7] == "1":
            K = 2
    elif pd_mev.iloc[i,7] == "2":
                k = 3
    elif pd_mev.iloc[i,7] == "3":
                    k = 4
    else:
        k = 5
    while k!= 0 and k!= 5 and j < pd_mev.iloc[i,5]:
        next_state = np.where(TPM[k]==max(TPM[k]))
        k = next_state[0][0]
        P = P*TPM
        j = j + 1
    pd_mev.iloc[i,-3] = j
    pd_mev.iloc[i,-2] = P[4,5]
    pd_mev.iloc[i,-1] = k

In [173]:
pd_mev.head()

,LoanID,MonthRep,LAST_RT,LAST_UPB,LoanAge,MonthsToMaturity,MaturityDate,CLDS,UNEMPLOY,HPI,CPALTT01USM661S,GS3M,GS5,GS10,CreditScore,DefaultF,DefaultP,DefaultS
0,100036401006,09/01/2017,5.750,163704.94,127,233,02/2037,1,-0.045455,0.005303,0.004607,0.019417,0.011236,0.011236,674.0,233,6.007550e-11,2
1,100048597640,09/01/2017,6.250,208724.20,126,234,03/2037,0,-0.045455,0.005303,0.004607,0.019417,0.011236,0.011236,762.0,2,8.713121e-01,0
2,100122275573,09/01/2017,6.125,148843.11,127,233,02/2037,0,-0.045455,0.005303,0.004607,0.019417,0.011236,0.011236,604.0,233,1.617806e-14,1
3,100134033573,09/01/2017,6.750,57125.92,128,52,01/2022,0,-0.045455,0.005303,0.004607,0.019417,0.011236,0.011236,740.0,2,8.592660e-01,0
4,100252434914,09/01/2017,5.375,18858.39,128,52,01/2022,0,-0.045455,0.005303,0.004607,0.019417,0.011236,0.011236,804.0,2,8.917865e-01,0


In [240]:
lgd_pd_temp = pd_mev[pd_mev['DefaultF'] != pd_mev['MonthsToMaturity'] ]

In [241]:
lgd_pd = lgd_pd_temp[~lgd_pd_temp['DefaultS'].isin([0,1])]

In [283]:
lgd_pd = lgd_pd.reset_index(drop=True)

In [247]:
lgd_pd['lgd'] = lgd_result.predict(lgd_pd)

In [288]:
lgd_pd['MP'] = lgd_pd['LAST_RT']/100/12*lgd_pd['LAST_UPB']*(1+lgd_pd['LAST_RT']/100/12)**lgd_pd['MonthsToMaturity']/((1+lgd_pd['LAST_RT']/100/12)**lgd_pd['MonthsToMaturity']-1)

In [289]:
lgd_pd['MP1'] = lgd_pd['MP'] *(1+lgd_pd['LAST_RT']/100/12)

In [295]:
lgd_pd['ead'] = lgd_pd['LAST_UPB']*(1+lgd_pd['LAST_RT']/100/12)**lgd_pd['DefaultF'] - lgd_pd['MP1']*((1+lgd_pd['LAST_RT']/100/12)**lgd_pd['DefaultF']-1)/(lgd_pd['LAST_RT']/1200)

In [297]:
lgd_pd['cecl'] = lgd_pd['ead']*lgd_pd['DefaultP']*lgd_pd['lgd']/((1+lgd_pd['LAST_RT']/100/12)**lgd_pd['DefaultF'])

In [304]:
result = sum(lgd_pd['cecl'])

In [305]:
print(result)

163592404.47582096


In [303]:
result / sum(pd_mev['LAST_UPB'])

0.042254805047209715

In [306]:
sum(pd_mev['LAST_UPB'])

3871569263.969986

In [307]:
lgd_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     111.6
Date:                Wed, 05 Dec 2018   Prob (F-statistic):           1.30e-71
Time:                        12:40:32   Log-Likelihood:                -3102.6
No. Observations:               18992   AIC:                             6213.
Df Residuals:                   18988   BIC:                             6245.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.8267      0.025     32.976      0.000       0.778       0.876
CreditScore    -0.0005   3.61e-05    -13.472      0.000      -0.001      -0.000
HPI             4.3948      0.379     11.593      0.000       3.652       5.138
UNEMPLOY        0.3829      0.096      3.970      0.000       0.194       0.572
==============================================================================
Omnibus:                     1029.423   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              791.170
Skew:                           0.409   Prob(JB):                    1.58e-172
Kurtosis:                       2.426   Cond. No.                     1.27e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.27e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""